```{contents}
```
## Thread Pool

A **thread pool** in LangGraph is a concurrency mechanism that allows **multiple nodes or graph invocations to execute simultaneously** by distributing work across a fixed set of worker threads.
It is a critical building block for achieving **high throughput, low latency, and scalable agent systems**.

---

### **1. Why Thread Pools Exist in LangGraph**

LLM workflows are naturally parallel:

* Multiple agents reason independently
* Tool calls wait on I/O
* Subgraphs execute concurrently
* Multiple user sessions run in parallel

Without controlled concurrency, systems either become **slow** or **unstable**.
LangGraph uses thread pools to provide **bounded, efficient parallel execution**.

---

### **2. Conceptual Model**

```
Incoming Graph Tasks
        |
   Task Scheduler
        |
   Thread Pool Executor
    |     |     |
 Worker  Worker  Worker
 Thread  Thread  Thread
    |     |     |
 Node   Node   Node
```

The thread pool:

* Limits resource usage
* Maximizes CPU utilization
* Prevents system overload

---

### **3. How LangGraph Uses Thread Pools**

LangGraph is built on **LangChain’s Runnable execution engine**, which uses Python’s `ThreadPoolExecutor` for parallel execution of nodes and subgraphs.

It is used for:

* Parallel node execution (`fan-out`)
* Concurrent tool calls
* Parallel agent execution
* Serving multiple graph invocations

---

### **4. Execution Flow with Thread Pool**

1. Graph compiled into execution plan
2. Scheduler identifies independent tasks
3. Tasks submitted to thread pool
4. Workers execute nodes concurrently
5. Results merged into shared state
6. Execution continues

---

### **5. Demonstration: Parallel Nodes**

```python
from concurrent.futures import ThreadPoolExecutor
from langgraph.graph import StateGraph, END
from typing import TypedDict
import time

class State(TypedDict):
    a: int
    b: int

def task1(state):
    time.sleep(2)
    return {"a": 1}

def task2(state):
    time.sleep(2)
    return {"b": 2}

builder = StateGraph(State)
builder.add_node("t1", task1)
builder.add_node("t2", task2)

builder.set_entry_point("t1")
builder.add_edge("t1", "t2")
builder.add_edge("t2", END)

graph = builder.compile()
graph.invoke({})
```

With proper parallel configuration, `task1` and `task2` can be executed concurrently in fan-out patterns.

---

### **6. Fan-Out / Fan-In with Thread Pool**

```
          Start
            |
        ┌───┴───┐
      Node A   Node B   ← parallel threads
        └───┬───┘
            |
          Join
```

This structure relies on thread pools for efficient execution.

---

### **7. Production Configuration Concepts**

| Parameter    | Purpose                    |
| ------------ | -------------------------- |
| max_workers  | Upper bound on concurrency |
| Queue size   | Prevent overload           |
| Task timeout | Avoid hung threads         |
| Retry policy | Recover from failures      |
| Backpressure | Slow input when saturated  |

Example:

```python
import concurrent.futures

executor = concurrent.futures.ThreadPoolExecutor(max_workers=8)
```

LangGraph internally uses similar configuration via runtime settings.

---

### **8. Thread Pool vs Async IO**

| Thread Pool             | Async IO             |
| ----------------------- | -------------------- |
| Good for blocking tasks | Best for network I/O |
| CPU-bound operations    | High scalability     |
| Simpler mental model    | More complex         |

LangGraph supports **both**, but thread pools are the default for safe concurrency.

---

### **9. Performance & Safety Guidelines**

* Never allow unbounded thread creation
* Size pool to CPU cores + I/O needs
* Isolate heavy agents in their own pools
* Use timeouts and retries
* Monitor thread saturation

---

### **10. Why This Matters for Agents**

Thread pools make it possible for LangGraph to run:

* Multi-agent systems
* Parallel tool execution
* High-throughput APIs
* Long-running autonomous workflows

**without deadlocks, resource exhaustion, or latency collapse**.



### Demonstration

In [2]:
# One-cell demonstration: Thread Pool style parallelism in LangGraph

from langgraph.graph import StateGraph, END
from typing import TypedDict
import time

# ----------------------------
# 1. Define Shared State
# ----------------------------

class State(TypedDict):
    x: int
    y: int
    z: int

# ----------------------------
# 2. Define Parallel Tasks
# ----------------------------

def start(state):
    return state

def task_a(state):
    print("Task A started")
    time.sleep(2)
    print("Task A finished")
    return {"x": 1}

def task_b(state):
    print("Task B started")
    time.sleep(2)
    print("Task B finished")
    return {"y": 2}

def join_task(state):
    print("Join started")
    return {"z": state["x"] + state["y"]}

# ----------------------------
# 3. Build Graph
# ----------------------------

builder = StateGraph(State)

builder.add_node("start", start)
builder.add_node("A", task_a)
builder.add_node("B", task_b)
builder.add_node("JOIN", join_task)

builder.set_entry_point("start")

# Fan-out: start triggers both A and B in parallel
builder.add_edge("start", "A")
builder.add_edge("start", "B")

# Fan-in: both A and B converge to JOIN
builder.add_edge("A", "JOIN")
builder.add_edge("B", "JOIN")

builder.add_edge("JOIN", END)

graph = builder.compile()

# ----------------------------
# 4. Run
# ----------------------------

start = time.time()
result = graph.invoke({})
end = time.time()

print("\nFinal State:", result)
print("Total Time:", round(end - start, 2), "seconds")


Task A started
Task B started
Task A finished
Task B finished
Join started

Final State: {'x': 1, 'y': 2, 'z': 3}
Total Time: 2.04 seconds
